# Parser 9.20 - Shortcut Assignment Operators

The goal of this version of the parser is to teach it to recognize shortcut assignment operators. That is, operators which perform an arithmetic operation using the value of the variable which appears their left side and the value of the expression on their right before assigning the final result to the variable on their left.

The reason they are called shortcuts is because they eliminate the need to specify the variable on both sides of the assignment. For example, both of these expressions add one to the value of **a**:

```Python
a = a + 1
a += 1
```

Though they have the same effect, the parsed Reverse Polish differs:

```Python
a = a + 1 => A A U* 1 B+ B=
a += 1    => A 1 B+=
```

The more compact shortcut assignment may also execute faster, since there is less overhead in the evaluation loop. In the end it still has to do everything the more verbose version does, but it can do so implicitly instead of explicitly.

## Libraries

In [ ]:
import glob       # for searching directories
import math       # for 'log--()' functions

import re         # for regular expressions

## User output

In [ ]:
visSep = '-------------'             # visual separator

def UIwriteln(this):
    '''write a single line to output'''
    print( f'{this}\n' )
    
def UIwriteSep():
    '''write a visual separator'''
    UIwriteln( visSep )

def UIshow(tag, value):
    '''write a tagged value to output'''
    UIwriteln( f'{tag}: {value}' )

def UIerror(this):
    '''write an error message to output'''
    UIshow( 'Error', this )

# Tracing

In [ ]:
# flags: show trace of processing

showInteract = True          # default for interactive use
showBatch = False            # default for batch use

showTrace = None             # control flag

# Trace Output

def TOshow(mesg, text):
    '''write trace message to output if enabled'''
    if showTrace:
        UIshow( f'{mesg:15s}', text )
        
def TOstring(tag, this):
    
    if showTrace:
        TOshow( tag, ' '.join([str(e) for e in this]) )

# -----------------------
# Parse Tracing
# -----------------------

def PTshowexpr(this):

    TOshow( 'Parse', visSep )
    TOshow( 'Current Expr', this )

def PTshowparse(ok, res, opStk, typStk):

    if ok:
        TOstring( 'Current RPN', res )
        TOstring( 'Operator Stack', opStk )
        TOstring( 'Type Stack', typStk )

def PTshowtoken(this):

    if not this[0] == ' ':
        TOshow( "Found Token", this )

# -----------------------
# Evaluation Tracing
# -----------------------

def ETshowtoken(this):
    
    TOshow( 'Eval', visSep )
    TOshow( 'Current token', this )

def ETshoweval(stk):
    
    TOstring( 'Operand Stack', stk )


# Parser

In [ ]:
# operands accepted:
# - decimal and hexadecimal floating point literals
# - scalar numeric variables

# operators accepted:
# - unary negation, plus
# - binary addition, subtraction, multiplication, division
# - grouping parentheses
# - logical not, equality, inequality
# - assignment and shortcut assignment

# errors detected:
# - unrecognized input
# - out of range numeric input
# - malformed expression

# result tuple:
# - (True, [parse])
# - (False, None)

class Parser(object):
    
    VERSIONNUMBER = '9.20'
    
    _FLTMAX =  4294967295                                  # 2**32-1
    _FLTMIN = -4294967296                                  # -(2**32)

    _expMax = {
        'P' : math.log2(_FLTMAX),                          # max base 2 exponent
        'E' : math.log10(_FLTMAX)                          # max base 10 exponent
    }

    _typeChkLst = {                                         # type check control lists
         'n2n': ['number', 'number'],                       # - index is left-to-right
        'nn2n': ['number', 'number', 'number'],             # check is right-to-left
        'vn2n': ['number', 'numsym', 'number']
    }
    
    def __init__(self):
        pass
        
    def doparse(self, this):

        def parseErr(mesg, pos):
            '''report parse error'''
            UIerror(mesg)
            UIwriteln(f'>>> {this}')
            if pos > 0:
                UIwriteln(f'{"^^near here".rjust(pos)}')
            return False

        # initialize

        expr = this                # save to new variable but retain original for error reports
        start = 15                 # tracked so we can report where in an expression an error occurred
        token = None               # anything successfully matched
        ok = wantoperand = True    # flags
        result = []                # rpn expression
        opStk = [ ('EOE', 1) ]     # operator stack
        typStk = []                # type stack
        

        def typeCheck(op, chk):
            '''type check operands'''            
            check = list(self._typeChkLst[chk])       # list() to avoid aliasing
            while len(check) > 1:
                want = check.pop()
                have, rpnPos, errPos = typStk.pop()
                              
                # do we want a number ?
                
                if want == 'number':
                    
                    # convert symbols to numbers
                    
                    if have == 'numsym':
                        result.insert( rpnPos, 'U*' )
                        
                # do we want a variable ?
                
                elif want == 'numsym':
                    
                    if have != 'numsym':
                        return parseErr( 'Numeric variable required', errPos )
                        
            # push intermediate result type (no position)
                    
            typStk.append( (check[0], -1, errPos) )
            return True
 
        def addOperand(op, typ):
            '''add operand to RPN'''
            result.append( op )
            typStk.append( (typ, len(result), start) )
            
        def addOperator():
            '''add operator to RPN'''            
            op, _, chk = opStk.pop()
            result.append( op )
            return typeCheck( op, chk )
            
        def popGEop(prec):
            '''pop operators of equal or greater precedence'''
            ok = True
            while ok and prec <= opStk[-1][1]:
                ok = addOperator()
            return ok
 
        def pushLeft(op, prec, chk):
            '''push left associative operator on stack'''
            if not popGEop(prec):
                return False
            opStk.append( (op, prec, chk) )
            return True

        def popGop(prec):
            '''pop operators of greater precedence'''
            ok = True
            while ok and prec < opStk[-1][1]:
                ok = addOperator()
            return ok
 
        def pushRight(op, prec, chk):
            '''push right associative operator on stack'''
            if not popGop(prec):
                return False
            opStk.append( (op, prec, chk) )
            return True

        def popUntil(op, prec):
            '''clear and check operator stack'''
            if not popGEop(prec):
                return False
            elif op == opStk.pop()[0]:
                return True
            elif op == '(':
                return parseErr('Unmatched right parenthesis', start)
            elif op == 'EOE':
                return parseErr('Unmatched left parenthesis', 0)      
    
        # operator dictionaries initialization
        
        _unOps = '[-+!]'

        _unOp =  {
#            '*': (100, pushRight, 'n2n'),
            '-':  (80, pushRight, 'n2n'),
            '+':  (80, pushRight, 'n2n'),
            '!':  (80, pushRight, 'n2n')
        }

        _binOps = '[-+*/]=?|[=!]?='

        _binOp = {
            '*':  (70, pushLeft,  'nn2n'),
            '/':  (70, pushLeft,  'nn2n'),
            '+':  (60, pushLeft,  'nn2n'),
            '-':  (60, pushLeft,  'nn2n'),
            '==': (50, pushLeft,  'nn2n'),
            '!=': (50, pushLeft,  'nn2n'),
            '=':  (10, pushRight, 'vn2n'),
            '*=': (10, pushRight, 'vn2n'),
            '/=': (10, pushRight, 'vn2n'),
            '+=': (10, pushRight, 'vn2n'),
            '-=': (10, pushRight, 'vn2n')
        }
      
        def convertFloat(fplit, base, capgrp):
            '''convert floating point literal to internal form'''
            
            def rangeErr():
                return parseErr(f'\'{fplit}\' is out of range', start)
            
            # collect the features of interest
                   
            p = re.search(capgrp, fplit.upper())
                            
            lint, lfrc, expbas, expsgn, lexp = p.group(1,2,4,5,6)
            
            # convert integer portion (if any)
 
            uint = 0
            if lint:
                p = re.search('[1-9A-F][0-9A-F]*', lint )
                if p != None:
                    for ch in p.group():
                        digval = '0123456789ABCDEF'.find(ch)
                        if uint <= (self._FLTMAX - digval)/base:
                            uint =  uint * base + digval
                        else:
                            return rangeErr()
                    
            # convert fractional portion (if any)
                    
            ufrc = 0
            if lfrc:
                fbase = 1
                for ch in lfrc:
                    digval = '0123456789ABCDEF'.find(ch)
                    fbase *= base
                    ufrc += digval/fbase
        
            if uint == self._FLTMAX and ufrc != 0:
                return rangeErr()
            
            # value so far
            
            uflt = uint + ufrc
 
            # convert exponent portion (if any)
            
            uexp = 0
            if lexp:
                for ch in lexp:
                    digval = '0123456789'.find(ch)
                    if uexp <= (self._expMax[expbas] - digval)/10:
                        uexp =  uexp * 10 + digval
                    else:
                        return rangeErr()
                    
            # adjust value by exponent (if any)
             
            if uexp:
                power = (2 if expbas == 'P' else 10) ** uexp
                if expsgn == '-':
                    uflt /= power
                elif uflt <= self._FLTMAX/power:
                    uflt *= power
                else:
                    return rangeErr()
                    
            addOperand( uflt, 'numlit' )
            return True

        def startsWith(regex):
            '''test if expression starts with given regular expression'''
            nonlocal expr, start, token

            p = re.match(regex, expr)
            if p == None:
                return False
            else:
                token = p.group()              # what we matched
                start += len(token)            # update to next match position in original string
                expr = expr[len(token):]       # "chop off" what we matched
                PTshowtoken(token)             # trace
                return True

        # top level main loop
              
        while ok and len(expr):

            _ = startsWith('[ ]+')                             # skip leading whitespace

            PTshowexpr(expr)                                   # trace

            # look for operand

            if wantoperand:

                if startsWith('[(]'):
                    '''left parenthesis ?'''
                    opStk.append( ('(', 2) )

                elif startsWith(_unOps):
                    '''right unary?'''
                    prec, assoc, check = _unOp[token]
                    assoc( 'U' + token, prec, check )

                else:

                    wantoperand = False                         # flip
                    
                    if startsWith('0[xX]([0-9a-fA-F]+([.][0-9a-fA-F]*)?|[.][0-9a-fA-F]+)([pP][-+]?[0-9]+)?'):
                        '''unsigned hexadecimal literal ?'''
                        ok = convertFloat(token, 16, '0X([0-9A-F]*)[.]?([0-9A-F]*)(([P])([-+])?([0-9]+))?')

                    elif startsWith('([0-9]+([.][0-9]*)?|[.][0-9]+)([eE][-+]?[0-9]+)?'):
                        '''unsigned decimal literal?'''
                        ok = convertFloat(token, 10, '([0-9]*)[.]?([0-9]*)(([E])([-+])?([0-9]+))?')
                        
                    elif startsWith('[a-zA-Z][_a-zA-Z0-9]*'):
                        '''numeric scalar variable?'''
                        addOperand( token.upper(), 'numsym')
                        
                    else:
                        '''malformed'''
                        ok = parseErr('Expecting operand', start)

            # look for operator

            else:

                if startsWith('[)]'):
                    ok = popUntil( '(', 4 )

                else:

                    wantoperand = True                                # flip
                    
                    if startsWith(_binOps):
                        '''binary operator?'''
                        prec, assoc, check = _binOp[token]
                        ok = assoc( 'B' + token, prec, check )

                    else:
                        '''malformed'''
                        ok = parseErr('Expecting operator', start)

            PTshowparse(ok, result, opStk, typStk )               # trace
 
        if ok and wantoperand:
            ok = parseErr('Unexpected end of expression', start ) # must be in 'wantoperator' state   

        if ok:
            ok = popUntil( 'EOE', 3 )                             # clear operator stack
            
        if ok:
            ok = typeCheck('LoneSymbol', 'n2n')                   # make sure final result is a number

        return (ok, result if ok else None)                       # done
       


### How it works

In theory changes to the parser should be confined to its initialization:

- extend the recognition of assignment operators to include four new shortcut assignments
- add their precedences to the precedence dictionary
- add their type check information to the type index dictionary
- and that's it

Alas, if only it were that easy! Doing this the most straightforward way suggests right associative binary operators could be recognized by a simple regular expression:

```Python
'[-+*/]?='
```

This basically means "an equals sign, optionally preceeded by one of four arithmetic operators". So it should match any of:

```Python
= *= /= += -=
```

...and it does. No problem there. However up to now left associative binary operators have been checked first. This picks off all the arithmetic operator prefixes of **'='**, since they are single character matches. Then the parser switches state and complains that the following **'='** sign is not an operand.

>You might expect that the matching engine in Python's regular expression library would search for the longest possible match, and avoid single character matches when two character matches are possible. Which is what other regex engines routinely do by default. But you'd be wrong.

Hmm. What if we reverse the order of checks so that right associative binary operators come first? Now our shortcut assignments work fine but we soon notice that our regex for them also matches

```Python
==
```

...but only the first **'='**. Then as before the parser complains that the second **'='** is not an operand.

>Regression testing. Useful but sometimes annoying.

If we want to keep our current arrangement of separate checks for left- and right-associative operators, we're going to have to find a way to express the idea of matching either "one but not two equal signs" or "an arithmetic operator *not* followed by an equal sign".

>The bright side of this approach is it allows us to extend our knowledge of Python regular expressions to include [lookaround assertions](https://www.regular-expressions.info/lookaround.html)! 

Let's start with the first possibility:

```Python
'[-+*/]=|(?<![=!])=(?![=])' 
```

This regex matches all our assignment operators without also matching **'=='**. Left of the **'|'**:

```Python
'[-+*/]='
```

matches any of our four shortcut assignment operators. Right of the **'|'**:

```Python
'(?<![=])=(?![=])'
```

matches **'='** but not **'=='**. It works by using *negative assertions* (note the **'!'** negation operator).

```Python
'(?<!...)'
```

is a *negative lookbehind assertion*. It matches if what follows it matches and is not preceded by whatever **'...'** matches. Since in our case we've used a character class consisting only of an **'='** sign, this matches if the **'='** following it is not preceded by another **'='**.

```Python
'(?!...)'
```

is a *negative lookahead assertion*. It matches if whatever precedes it is not followed by whatever **'...'** matches. In our case, we've specified that we do not want an **'='** sign to follow another **'='** sign.

>This is actually a general solution to the problem of matching one character but not two of them in a row. Just replace **'='** with whatever character should appear only once.

This is not the easiest regex to understand at first sight, but it does work in our regression tests if we check for right associative operators first.

What about the other idea?

```Python
'[-+*/](?![=])|==|!='
```

This uses only one negative assertion to match an arithmetic operator that is *not* followed by an **'='** sign. This also passes all our regression tests if we check for left associative operators first.

These are somewhat ugly solutions, though. Either regex is complicated by what is essentially special case checking. This may become even more of a nuisance as more operators are added. So instead we'll do something else that attempts to preserve more of the simplicity we're looking for.

We'll start by discarding the idea of separately checking for left- and right-associative binary infix operators. The reason we've been doing so to this point is because we have to treat them differently once we've found them. But we can accomplish that purpose another way.

We can use a dictionary indexed by operator to hold everything we need to know about that operator - its precedence, associativity and type checks. This in itself is a simplification of what we've been doing so far.

We could put a flag in the dictionary to indicate left- or right-associativity. That would work; we'd call *pushLeft()* or *pushRight()* with the operator and its precedence depending on the flag. The code might even be a little cute:

```Python
if startsWith(_binOps):
    '''binary operator?'''
    prec, assoc, check = _binOp[token]
    ok = pushLeft('B'+token, prec, check) if assoc == 'Left' else pushRight('B'+token, prec, check)
```       
   
However we can also put the *pushLeft()* and *pushRight()* functions into the dictionary directly and call them indirectly. Which seems even cuter:

```Python
if startsWith(_binOps):
    '''binary operator?'''
    prec, assoc, check = _binOp[token]
    ok = assoc('B'+token, prec, check)
```       

>We have extended *pushLeft()* and *pushRight()* to accept and save an operator's type check information on the operator stack. Plus we create a similar dictionary for unary operators.

The only hitch in this approach is exactly the same one we've encountered before in the evaluator: the problem of forward reference to the functions we want to call indirectly. Here those functions are nested within *doparse()*, where initialization of a *Parser* object cannot "see" them. We solve it the same way, by moving the new dictionaries inside *doparse()* and placing them after the definitions of the functions we'll call indirectly.

We now need only relatively compact regexes to specify all the operators we currently recognize. Since they do not contain any forward references they could be placed outside *doparse()*, but they're easier to understand when near the operator dictionaries.

# Evaluator

In [ ]:
# operators handled:
# - unary negation, plus
# - binary addition, subtraction, multiplication, division
# - logical negation, equality and inequality
# - variable name de-reference
# - variable assignment and shortcut assignment

# errors detected:
# - out of range
# - division by zero

# return tuple:
# - (True, result)
# - (False, None)

class Evaluator(Parser):
    
    def __init__(self):
        self._symTable = dict()
 
    def doeval(self, rpn):
 
        def inRange(ok, val):
            '''range check test result'''
            if ok:
                stk.append( val )
            else:
                UIerror( 'Evaluation result out of range' )
            return ok
        
        def pushOperand(val):
            '''push operand on stack'''
            stk.append( val )
            return True
                     
        def binVal(val, var):
            '''assign value to variable'''
            self._symTable[var] = val
            return pushOperand(val)
        
        def unVal(var):
            '''variable name value'''
            return pushOperand(self._symTable[var] if var in self._symTable else 0)

#        def unPlu(arg):
#           '''unary plus'''
#           return pushOperand( arg )

        def unNeg(arg):
            '''unary negation'''
            return inRange( arg != self._FLTMIN, -arg )
        
        def unNot(arg):
            '''logical not'''
            return pushOperand( not arg )

        def binAdd(rgt, lft):
            '''binary addition'''
            if lft >= 0:
                return inRange( rgt <= self._FLTMAX - lft, lft+rgt )       
            else:
                return inRange( rgt >= self._FLTMIN - lft, lft+rgt )
 
        def binSub(rgt, lft):
            '''binary subtraction'''
            if lft >= 0:
                return inRange( lft - self._FLTMAX <= rgt, lft-rgt )
            elif rgt >= 0:
                return inRange( lft - self._FLTMIN >= rgt, lft-rgt )
            else:
                return inRange( lft <= self._FLTMAX + rgt, lft-rgt )
 
        def binMul(rgt, lft):
            '''binary multiplication'''
            if abs(lft) <= 1 or abs(rgt) <= 1:
                return pushOperand( lft * rgt )

            elif lft > 0:
                if rgt > 0:
                    return inRange( rgt <= self._FLTMAX / lft, lft * rgt )
                else:
                    return inRange( rgt >= self._FLTMIN / lft, lft * rgt )

            elif rgt > 0:
                return inRange( rgt <= self._FLTMIN / lft, lft * rgt )
  
            else:
                return inRange( rgt >= self._FLTMAX / lft, lft * rgt )
 
        def binDiv(rgt, lft):
            '''binary division'''
            if abs(rgt) >= 1:
                return pushOperand( lft/rgt )
            
            elif rgt > 0:
                if lft > 0 :
                    return inRange( lft <= self._FLTMAX * rgt, lft / rgt )
                else:
                    return inRange( lft >= self._FLTMIN * rgt, lft / rgt )
            
            elif rgt < 0:
                if lft > 0:
                    return inRange( lft <= self._FLTMIN * rgt, lft / rgt )
                else:
                    return inRange( lft >= self._FLTMAX * rgt, lft / rgt )

            else:
                UIerror( 'Division by zero' )
                return False
            
        def binEqu(rgt, lft):
            '''logical equality'''
            return pushOperand( lft == rgt )
        
        def binNeq(rgt, lft):
            '''logical inequality'''
            return pushOperand( lft != rgt )
        
        def binShortVal(val, var, op):
            '''shortcut assignment'''
            _ = unVal(var)                           # put the value of the variable on the stack
            if op(val, stk.pop()):                   # perform the arithmetic
                return binVal(stk.pop(), var)        # if successful,  also assign result to variable
            return False
        
        def binAddVal(val, var):
            return binShortVal(val, var, binAdd)
        
        def binSubVal(val, var):
            return binShortVal(val, var, binSub)
        
        def binMulVal(val, var):
            return binShortVal(val, var, binMul)
        
        def binDivVal(val, var):
            return binShortVal(val, var, binDiv)
        
        # initialize
                                
        unDispatch = {
            'U*': unVal,
            'U-': unNeg,
            'U+': pushOperand,
            'U!': unNot
        }
        
        binDispatch = {
            'B+': binAdd, 'B-': binSub,
            'B+=': binAddVal, 'B-=': binSubVal,
            'B*': binMul, 'B/': binDiv,
            'B*=': binMulVal, 'B/=': binDivVal,
            'B==': binEqu, 'B!=': binNeq,
            'B=': binVal
        }
  
        stk = []
        ok = True

        # main loop
        
        TOstring('Input', rpn)
        
        for v in rpn:

            ETshowtoken(v)
            
            if v in binDispatch:
                ok = binDispatch[v](stk.pop(), stk.pop())
                
            elif v in unDispatch:
                ok = unDispatch[v](stk.pop())
                
            else:
                stk.append( v )

            if not ok:
                return ( False, None )

            ETshoweval( stk )

        return ( True, stk.pop() )


### How it works

We add four new binary operators to *binDispatch{}'*. The functions which implement them are all so similar, differing only in which specific arithmetic operation they perform, that we use *binShortVal()*  to actually do the work. We pass the name of the function that performs that operation as a parameter along with the variable name and the value of the right side of the assignment.

Inside *binShortVal()* we leverage existing functions we already have. This leads to some small inefficiencies, most notably in the call to *unVal()* to retrieve the value of the left side variable. The actual value is left on the operand stack (which is not really where we want it) and the return value is always **True** (which we don't care about). So we have to pop the value off the stack in order to pass it to the relevant arithmetic function.

>If we instead re-implemented the part of *unVal()* that retrieves the value within *binShortVal()* itself, we could avoid any intermediate use of the operand stack. Overall the function size would increase and it would execute slightly faster. However if any changes were ever made to how *unVal()* looks up values, we'd have to remember to make them in *binShortVal()* as well. Just the sort of thing which is easy to overlook and leads to bugs that may be difficult to track down.

What do we gain in exchange for these inefficiencies? Two things: smaller code size, and more importantly, the error checks already built into the leveraged functions.

## Running the parser

In [ ]:
passCnt = failCnt = 0                       # most useful for test input files, but never any harm

myParser = myEvaluator = None               # where we keep instances of our classes

def startUp(flag):
    '''begin execution'''
    global passCnt, failCnt, showTrace
    global myParser, myEvaluator 
    if not myParser:
        myParser = Parser()
    if not myEvaluator:
        myEvaluator = Evaluator()
    UIshow( 'Parser', myParser.VERSIONNUMBER )
    passCnt = failCnt = 0
    showTrace = flag
     
def shutDown():
    '''terminate execution'''
    UIwriteSep()
    UIshow( 'Pass', passCnt )
    UIshow( 'Fail', failCnt )
    
# run parser

def parseOne(this):
    '''parse/evaluate one expression'''
    global passCnt, failCnt
    UIwriteSep()
    neg = this[0] == '@'
    if neg:
        this = this[1:]
    UIshow( 'Input', this )
    ok, res = myParser.doparse( this )
    if ok:
        UIshow( 'Final Parse', res )
        ok, res = myEvaluator.doeval( res )
        if ok:
            UIshow( 'Final Eval', res )
        if neg:
            ok = not ok
    if ok:
        passCnt += 1
    else:
        failCnt += 1

We now have a testing difficulty we have not previously encountered. Range testing shortcut operators require at least two expressions. First we must successfully assign some non-zero value to a variable. Only then can we use a shortcut operator to try pushing that value past a limit.

The problem of course is that our fail tests have so far been designed so that every expression fails. A successful assignment counts as a pass, and thus (somewhat paradoxically) will cause our fail test to fail.

Therefore we introduce a new convention: if the first character of a test expression is '@', the success/fail flag returned by the evaluator is negated. A successful result will be displayed, but will not be counted as a pass.

>Contrawise, a negated failed result will be counted as a pass. Something we may need to watch out for in the future.

## Interactive use

In [ ]:
def parse():
    
    startUp(showInteract)
    while True:
        inp = input( 'Expression: ' )
        UIwriteln( '' )                      # looks better with a blank line here
        if inp.upper()[0] == 'Q':
            break
        elif inp.strip():
            parseOne( inp )
    shutDown()

## Batch processing

In [ ]:
testDir = '..\\ParserTest\\'            # directory holding test input files (empty string if same as notebook directory)

# convert current version number to match test file numbers
# - done this way so we can update only the version number and everything still works

def currNum():
    
    head = myParser.VERSIONNUMBER[:len(myParser.VERSIONNUMBER)-3]
    tail = myParser.VERSIONNUMBER[-2:]
    return f'{head:0>2}{tail}'

# make full path name to test file

def makePath(typ, num):
    return f'{testDir}{typ}{num}.txt'

# run one test

def runTest(this):
    
    UIwriteln(f'Parser {myParser.VERSIONNUMBER} vs {this[-12:-4]}')
    
    myEvaluator._symTable.clear()
    
    with open(this) as f:
        data = f.readlines()
    for line in data:
        test = line.strip()
        if test and test[0] != '#':         # skip blank and comment lines
            parseOne(test)
    
# run a test of current or specified version which should succeed
    
def good(num='curr'):
  
    startUp(showBatch)
    runTest(makePath('pass', currNum() if num == 'curr' else num))
    shutDown()
    
# run a test of current or specified version which should fail

def bad(num='curr'):
    
    startUp(showBatch)
    runTest(makePath('fail', currNum() if num == 'curr' else num))
    shutDown()
    
# run regression test against current and all previous test files

def regress():
            
    UIwriteln('PASS tests')
    
    startUp(showBatch)                       # must create objects before we can access variables inside them 
    currFn = makePath('pass', currNum())
    failed = []
    fnlist = glob.glob(f'{testDir}pass????.txt')
    for fn in fnlist:
        if fn <= currFn:
            atstart = failCnt
            runTest(fn)
            if atstart < failCnt:
                failed.append(fn)               
    shutDown()
    
    UIwriteln('FAIL tests')
    
    startUp(showBatch)
    currFn = makePath('fail',currNum())
    passed = []
    fnlist = glob.glob(f'{testDir}fail????.txt')
    for fn in fnlist:
        if fn <= currFn:
            atstart = passCnt
            runTest(fn)
            if atstart < passCnt:
                passed.append(fn)                
    shutDown()
    
    if not len(failed):
        UIwriteln('All pass tests succeded')
    else:
        UIwriteln('Pass tests which failed')
        for fn in failed:
            UIwriteln(f'  {fn}')
            
    if not len(passed):
        UIwriteln('All fail tests succeded')
    else:
        UIwriteln('Fail tests which passed')
        for fn in passed:
            UIwriteln(f'   {fn}')
              

# Testing the parser

In [ ]:
parse()       # interactive, one expression at a time

In [ ]:
good()        # run current parser against its own pass test. Use good('1234') to run against specific pass test.

In [ ]:
bad()         # run current parser against its own fail test. Use bad('5678') to run against specific fail test.

In [ ]:
regress()     # run parser against all previous tests